In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("drive/MyDrive/data for colab/x_train.csv", sep=",")
test = pd.read_csv("drive/MyDrive/data for colab/y_train.csv", sep=",")
contest_data_id = pd.read_csv("drive/MyDrive/data for colab/x_test.csv", sep=",")

In [ ]:
data

,id,cat,cow,dog,horse,sheep,bounds_x,bounds_y,bounds_width,bounds_height,...,L_F_paw_y,R_F_paw_v,R_F_paw_x,R_F_paw_y,L_B_paw_v,L_B_paw_x,L_B_paw_y,R_B_paw_v,R_B_paw_x,R_B_paw_y
0,265,0,1,0,0,0,37,16,240,170,...,167,1,161,168,1,94,173,1,55,177
1,676,0,0,0,1,0,43,8,208,236,...,213,1,193,209,1,77,232,1,115,233
2,441,0,0,1,0,0,65,0,234,222,...,0,0,0,0,0,0,0,0,0,0
3,328,0,1,0,0,0,15,8,272,178,...,171,1,190,172,1,52,176,1,35,178
4,102,1,0,0,0,0,2,1,148,146,...,130,1,68,131,1,3,128,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,356,0,1,0,0,0,15,1,266,188,...,0,0,0,0,0,0,0,0,0,0
672,960,0,0,0,0,1,123,72,702,580,...,615,1,618,617,1,172,599,1,223,609
673,812,0,0,0,0,1,57,12,176,176,...,170,1,171,175,1,80,172,1,107,176
674,132,1,0,0,0,0,19,8,278,214,...,218,1,130,219,1,290,106,1,260,121


In [ ]:
test

,id,nothing,stand,sit,lie,go,run
0,265,0,0,0,0,1,0
1,676,0,0,0,0,1,0
2,441,1,0,0,0,0,0
3,328,0,1,0,0,0,0
4,102,0,0,0,1,0,0
...,...,...,...,...,...,...,...
671,356,0,1,0,0,0,0
672,960,0,1,0,0,0,0
673,812,0,1,0,0,0,0
674,132,0,0,0,1,0,0


In [ ]:
(data.dtypes != "int64").any()

False

In [ ]:
data.isna().sum().any()

False

In [ ]:
X = data.drop(columns="id")
y = test.drop(columns="id")
contest_data = contest_data_id.drop(columns=["id"])

In [ ]:
labels = {"nothing": 0, "stand": 1, "sit": 2, "lie": 3, "go": 4, "run": 5}
y = y.idxmax(axis=1).map(labels)
y

0      4
1      4
2      0
3      1
4      3
      ..
671    1
672    1
673    1
674    3
675    0
Length: 676, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Дерево решения

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier(min_samples_split=len(labels), random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_valid)

report = classification_report(
    y_valid,
    y_pred,
    labels=list(labels.values()),
    target_names=labels.keys()
)
print(report)

              precision    recall  f1-score   support

     nothing       0.54      0.47      0.50        15
       stand       0.76      0.69      0.72        83
         sit       0.43      0.30      0.35        10
         lie       0.13      0.20      0.16        10
          go       0.08      0.20      0.12         5
         run       0.64      0.69      0.67        13

    accuracy                           0.58       136
   macro avg       0.43      0.42      0.42       136
weighted avg       0.63      0.58      0.60       136



In [ ]:
model = DecisionTreeClassifier(min_samples_split=len(labels), random_state=42)
model.fit(X, y)
label_ans = model.predict(contest_data)

ans = pd.DataFrame({"id": contest_data_id["id"], "label": label_ans})
# ans.to_csv("ans.csv", index=False)
ans.head()

,id,label
0,582,5
1,42,1
2,148,1
3,737,5
4,213,3


Kaggle score: 0.53448

# Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

scores = []
for n_estimators in range(100, 1000, 100):
  model = RandomForestClassifier(
      n_estimators=n_estimators,
      max_depth=20,
      min_samples_split=len(labels),
      random_state=42
  )
  model.fit(X_train, y_train)

  y_pred = model.predict(X_valid)

  scores.append(f1_score(y_valid, y_pred, average='micro'))
print(scores)

[0.6985294117647058, 0.7058823529411765, 0.7132352941176471, 0.7205882352941176, 0.7279411764705882, 0.7205882352941176, 0.7205882352941176, 0.7279411764705882, 0.7279411764705882]


In [ ]:
model = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=len(labels),
    random_state=42
)
model.fit(X_train, y_train)

y_pred = model.predict(X_valid)

report = classification_report(
    y_valid,
    y_pred,
    labels=list(labels.values()),
    target_names=labels.keys()
)
print(report)

              precision    recall  f1-score   support

     nothing       0.79      0.73      0.76        15
       stand       0.73      0.93      0.82        83
         sit       0.50      0.30      0.37        10
         lie       0.71      0.50      0.59        10
          go       0.00      0.00      0.00         5
         run       1.00      0.23      0.38        13

    accuracy                           0.73       136
   macro avg       0.62      0.45      0.49       136
weighted avg       0.72      0.73      0.69       136



In [ ]:
model = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=len(labels),
    random_state=42
)
model.fit(X, y)
label_ans = model.predict(contest_data)

ans = pd.DataFrame({"id": contest_data_id["id"], "label": label_ans})
# ans.to_csv("ans3.csv", index=False)
ans.head()

,id,label
0,582,1
1,42,1
2,148,1
3,737,1
4,213,0


Kaggle score: 0.51724

# Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=len(labels),
    random_state=42
)
model.fit(X_train, y_train)

y_pred = model.predict(X_valid)

report = classification_report(
    y_valid,
    y_pred,
    labels=list(labels.values()),
    target_names=labels.keys()
)
print(report)

              precision    recall  f1-score   support

     nothing       0.77      0.67      0.71        15
       stand       0.74      0.84      0.79        83
         sit       0.40      0.20      0.27        10
         lie       0.57      0.80      0.67        10
          go       0.17      0.20      0.18         5
         run       0.75      0.23      0.35        13

    accuracy                           0.69       136
   macro avg       0.57      0.49      0.50       136
weighted avg       0.69      0.69      0.67       136



In [ ]:
model = GradientBoostingClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=len(labels),
    random_state=42
)
model.fit(X, y)
label_ans = model.predict(contest_data)

ans = pd.DataFrame({"id": contest_data_id["id"], "label": label_ans})
# ans.to_csv("ans4.csv", index=False)
ans.head()

,id,label
0,582,1
1,42,5
2,148,1
3,737,4
4,213,0


Kaggle score: 0.51724